### Extract features from the processed and augmented audio files.

In [8]:
# Imports
import os
import random
import logging
import warnings
import gc

import numpy as np
import pandas as pd
from tqdm import tqdm


# Data processing and scientific computing
from scipy.io import wavfile
from scipy.signal import butter, lfilter
from scipy.spatial.distance import cosine

# Audio processing
import librosa
import soundfile as sf

# Visualization
import matplotlib.pyplot as plt
import plotly.express as px

# Set up logging
logging.basicConfig(level=logging.INFO)

In [9]:
# Load the prepared csv
augmented_data = pd.read_csv("augmented_data.csv")

In [10]:
augmented_data.head()

,genus,species,latitude,longitude,quality,file_name,simplified_type,season,time_of_day,length_seconds,processed_file,augmentations
0,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN
1,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN
2,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN
3,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN
4,Branta,canadensis,39.2095,-84.7821,A,Branta_canadensis_Whitewater_Township_near__Ha...,Call,Spring,Morning,14,Branta_canadensis_Whitewater_Township_near__Ha...,NaN


In [11]:
def is_valid_audio(audio, sr, min_duration=0.1, silence_threshold=-60):
    """Check if the audio segment is valid (not too short and not silent)."""
    duration = librosa.get_duration(y=audio, sr=sr)
    if duration < min_duration:
        return False

    # Check if the audio is mostly silent
    db = librosa.amplitude_to_db(np.abs(audio), ref=np.max)
    if np.mean(db) < silence_threshold:
        return False

    return True

def extract_features(audio, sr):
    # Mel-spectrogram
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(audio)[0]
    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    
    return {
        'mel_spectrogram_db': mel_spec_db,
        'mfccs': mfccs,
        'spectral_centroids': spectral_centroids,
        'chroma': chroma,
        'zero_crossing_rate': zero_crossing_rate,
        'spectral_rolloff': spectral_rolloff
    }

def summarize_feature(feature):
    if feature.ndim == 1:
        return np.array([np.mean(feature), np.std(feature), np.max(feature)])
    elif feature.ndim == 2:
        return np.hstack([
            np.mean(feature, axis=1),
            np.std(feature, axis=1),
            np.max(feature, axis=1)
        ])
    else:
        raise ValueError(f"Unsupported feature dimension: {feature.ndim}")

def save_mel_spectrogram(mel_spec, output_dir, base_filename, sr):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spec, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel-spectrogram')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"{base_filename}_mel_spectrogram.png"))
    plt.close()


In [12]:
def process_audio_file(file_path, output_dir):
    try:
        # Load audio file
        audio, sr = librosa.load(file_path, sr=None)
        
        # Check if audio segment is valid
        if not is_valid_audio(audio, sr):
            print(f"Warning: Audio file {file_path} is too short or silent. Skipping.")
            return None, None

        # Extract features
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            features = extract_features(audio, sr)
        
        # Summarize features
        feature_summary = {}
        for key, value in features.items():
            if key != 'mel_spectrogram_db':
                feature_summary[f"{key}_summary"] = summarize_feature(value)
        
        # Create feature vector
        feature_vector = np.hstack([
            feature_summary.get('mfccs_summary', np.array([])),
            feature_summary.get('spectral_centroids_summary', np.array([])),
            feature_summary.get('chroma_summary', np.array([])),
            feature_summary.get('zero_crossing_rate_summary', np.array([])),
            feature_summary.get('spectral_rolloff_summary', np.array([]))
        ])
        
        # Save mel-spectrogram as image
        base_filename = os.path.splitext(os.path.basename(file_path))[0]
        save_mel_spectrogram(features['mel_spectrogram_db'], output_dir, base_filename, sr)
        
        return feature_vector, features
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return None, None

def process_audio_files_batch(df, base_dir, batch_size=500):
    feature_data = []
    skipped_files = []
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        batch_feature_data = []  # Temporary storage for batch features
        for _, row in tqdm(batch.iterrows(), total=len(batch)):
            possible_paths = [
                os.path.join(base_dir, 'Augmented Recordings', row['processed_file']),
                os.path.join(base_dir, 'Processed Recordings', row['processed_file'])
            ]
            
            file_path = next((path for path in possible_paths if os.path.exists(path)), None)
            
            if file_path is None:
                print(f"File not found: {row['processed_file']}")
                skipped_files.append(row['processed_file'])
                continue
            
            feature_vector, features = process_audio_file(file_path, output_dir="mel-spectrograms")
            
            if feature_vector is not None:
                batch_feature_data.append({
                    'processed_file': row['processed_file'],
                    'feature_vector': feature_vector,
                })
            else:
                skipped_files.append(row['processed_file'])
            
            # More frequent garbage collection to avoid memory overflow
            gc.collect()
        
        # Append batch data to main feature data and save to disk
        feature_data.extend(batch_feature_data)
        batch_df = pd.DataFrame(batch_feature_data)
        batch_df.to_csv(f"feature_data_batch_{i//batch_size}.csv", index=False)
        
        # Clear batch data from memory
        batch_feature_data.clear()
        gc.collect()

    print(f"Total files skipped: {len(skipped_files)}")
    return feature_data, skipped_files


In [13]:
# Usage

base_dir = os.getcwd()
output_dir = 'mel-spectrograms'

feature_data, skipped_files = process_audio_files_batch(augmented_data, base_dir)

# Convert feature_data to DataFrame
feature_df = pd.DataFrame(feature_data)

# Merge the new feature DataFrame with the existing final_data DataFrame
final_data = pd.merge(augmented_data, feature_df, on='processed_file', how='left')

# Drop rows corresponding to skipped files
final_data = final_data[~final_data['processed_file'].isin(skipped_files)]

print(final_data.info())

print(f"\nTotal files in augmented_data: {len(augmented_data)}")
print(f"Files successfully processed: {len(feature_df)}")
print(f"Files skipped (too short or silent): {len(skipped_files)}")
print(f"Files in final_data after dropping skipped files: {len(final_data)}")

Testing feature extraction on file: Augmented Recordings\Setophaga_magnolia_Magee_Marsh_-_boardwalk_Lucas_County_Ohio_828025_segment_5_aug_3.wav
mel_spectrogram_db shape: (128, 453)
mfccs shape: (13, 453), mean: -7.0931, std: 97.3944
spectral_centroids shape: (453,), mean: 2883.1173, std: 1035.0546
chroma shape: (12, 453), mean: 0.8048, std: 0.1477
zero_crossing_rate shape: (453,), mean: 0.0173, std: 0.0195
spectral_rolloff shape: (453,), mean: 6860.2028, std: 2504.3488

Summarized Features:
mfccs_summary type: numpy.ndarray, shape: (39,), mean: 4.5996, std: 70.4689
spectral_centroids_summary type: numpy.ndarray, shape: (3,), mean: 4322.4834, std: 3426.4439
chroma_summary type: numpy.ndarray, shape: (36,), mean: 0.6488, std: 0.3681
zero_crossing_rate_summary type: numpy.ndarray, shape: (3,), mean: 0.0705, std: 0.0737
spectral_rolloff_summary type: numpy.ndarray, shape: (3,), mean: 9122.1031, std: 6525.8239

Final feature vector shape: (84,)
Mel-spectrogram saved as: Setophaga_magnolia_

In [7]:
# Usage

base_dir = os.getcwd()
output_dir = 'mel-spectrograms'

feature_data, skipped_files = process_audio_files_batch(augmented_data, base_dir)

# Convert feature_data to DataFrame
feature_df = pd.DataFrame(feature_data)

# Merge the new feature DataFrame with the existing final_data DataFrame
final_data = pd.merge(augmented_data, feature_df, on='processed_file', how='left')

# Drop rows corresponding to skipped files
final_data = final_data[~final_data['processed_file'].isin(skipped_files)]

print(final_data.info())

print(f"\nTotal files in augmented_data: {len(augmented_data)}")
print(f"Files successfully processed: {len(feature_df)}")
print(f"Files skipped (too short or silent): {len(skipped_files)}")
print(f"Files in final_data after dropping skipped files: {len(final_data)}")

  3%|▎         | 26/1000 [00:07<04:52,  3.33it/s]

 11%|█         | 111/1000 [00:32<04:05,  3.63it/s]

 12%|█▏        | 120/1000 [00:35<04:51,  3.02it/s]

 15%|█▍        | 147/1000 [00:42<03:59,  3.56it/s]

 17%|█▋        | 167/1000 [00:48<04:24,  3.15it/s]

...

 86%|████████▌ | 857/1000 [07:02<01:48,  1.32it/s]

 94%|█████████▍| 945/1000 [08:09<00:40,  1.36it/s]

 96%|█████████▌| 958/1000 [08:19<00:31,  1.33it/s]

100%|█████████▉| 997/1000 [08:47<00:02,  1.34it/s]

100%|██████████| 1000/1000 [08:49<00:00,  1.89it/s]

In [ ]:
# Review the dataframe
final_data.info()

In [ ]:
# Save DataFrame to CSV
final_data.to_csv('final_data.csv', index=False)

print("Processing complete. Summary data saved to 'final_data.csv'.")